### Run VOLTTRON interface testing  


### Selecting a  Control and the Test Case 


In [6]:
import os
import sys
sys.path.append("..")

from Volttron_interface_os_c_1a_zone_temp_adj_inc_ratch import (
    VolttronControls,
    VolttronControlFunctions,
    VolttronInterface
)

# Select control and test case identifier 
control = 'os_c_1a_zone_temp_adj_inc_ratch' 

# os_c_1a_zone_temp_adj_inc_ratch
# os_c_1b_zone_temp_adj_perform_ratch
# os_c_2a_zone_pre_heat_cool_sim
# os_c_2b_zone_pre_heat_cool_com
# os_c_1a2a_zone_temp_adj_inc_ratch_pre_heat_cool_sim
# os_c_1a2b_zone_temp_adj_inc_ratch_pre_heat_cool_com
# os_c_1b2a_zone_temp_adj_perform_ratch_pre_heat_cool_sim
# os_c_1b2b_zone_temp_adj_perform_ratch_pre_heat_cool_com
# os_c_3_chi_water_temp_reset

testcase = 'testcase_wcec' 
graph_path = os.path.join(os.getcwd(), f'{testcase}', f'{testcase}.ttl')

### Create Application 
Create application based on the selected control. 

In [7]:
from utility_functions.get_price_threshold import get_price_threshold

class TestCaseInterface(VolttronInterface):
    def __init__(self, controls, config_path):
        super().__init__(controls, config_path)
    
    def get_price_threshold(self, *args):
        return get_price_threshold(*args)
    
def create_application(config_path) -> VolttronInterface:
    '''takes a config file and returns an instantiated TestCaseInterface
    
    Parameters
    ----------
        config_path : str
            the path to a configuration file for instantiating a bestest air shed application 
    Returns
    -------
        application : TestCaseInterface
            returns an instantiated bestest air interface with shedding controls
'''
    
    functions = VolttronControlFunctions()
    controls = VolttronControls(functions)
    
    # Instantiate the TestCaseInterface with the controls, config_path, and hvac_signal_function
    application = TestCaseInterface(controls, config_path)
    return application


### Instantiate Application 
Instantiate the test case interface with the selected control.

In [8]:
import os
config_path = os.path.join(os.getcwd(), f'{testcase}', f'config_{control}.yml')

app = create_application(config_path)

# print of the control's SPARQL result

(rdflib.term.URIRef('urn:bldg/zone_x'), rdflib.term.Literal('temp_x'), None, None, rdflib.term.Literal('occ_x'), None, None, rdflib.term.Literal('setpoint_x'), None, None, None, None)


### Defining the Testing Parameters
Defining testing parameters for the first hour of the day

In [9]:
from datetime import datetime
now = datetime.now()

step = 1 # 1 for 1h; 0.25 for 15min and so on
current_time = now.hour + now.minute / 60  # Current hour + fraction of minutes
operation_mode = 'heat'

y = {
       'temp_x': 19,
       'setpoint_x': 21
    }

output = app.control_agent(step, current_time, operation_mode, y)

print(f"Output at time {current_time}: {output}")

9.716666666666667
{'temp_x': 19, 'setpoint_x': 21}
{}
price_threshold_value 0.2666
range(0, 1) ['urn:bldg/zone_x'] ['temp_x'] ['setpoint_x'] [] [] [] [] ['occ_x'] [] [] [] None None None
heat
TSetHeaZon 21 TSetHeaZon_baseline [21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 21.0, 21.0]
TSetCooZon None TSetCooZon_baseline []
TZon 19
occupancy_schedule [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
price_schedule [0.2666, 0.4, 0.6, 0.8, 0.9, 0.9666, 0.2666, 0.2666, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2666]
0 0
qualified zone
first shed heat
{}
new shed counter 1
{'setpoint_x': 20}
ratchet []
rebound []
rebound []
[]
[]
[]
{'setpoint_x': 20}
Output at time 9.716666666666667: {'setpoint_x': 20}


Run the control for the remaining hours of the day

In [10]:
total_hours = 24  # Run for 24 hours
total_steps = int(total_hours / step)  # 96 steps (24 * 4)

# Initialize previous values for target temperatures
prev_setpoint_x = None

for i in range(total_steps): 
    current_time += step  # Increment by 15 minutes    

    if current_time >= 24:
        current_time -= 24
    
    # Get the target temperature values, default to the previous value if not available
    setpoint_x = output.get('setpoint_x', prev_setpoint_x)

    y = {
        'temp_x': 19,
        'setpoint_x': setpoint_x,
    }

    output = app.control_agent(step, current_time, operation_mode, y)

    print(f"Output at time {current_time}: {output}")

    # Update previous values for the next iteration
    prev_setpoint_x = setpoint_x


10.716666666666667
{'temp_x': 19, 'setpoint_x': 20}
{0: 1}
price_threshold_value 0.2666
range(0, 1) ['urn:bldg/zone_x'] ['temp_x'] ['setpoint_x'] [] [] [] [] ['occ_x'] [] [] [] None None None
heat
TSetHeaZon 20 TSetHeaZon_baseline [21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 21.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 16.0, 21.0, 21.0, 21.0]
TSetCooZon None TSetCooZon_baseline []
TZon 19
occupancy_schedule [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]
price_schedule [0.4, 0.6, 0.8, 0.9, 0.9666, 0.2666, 0.2666, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2383, 0.2666, 0.2666]
0 1
qualified zone
heating season TSetHeaZon 20
True
ratcheting shed heat
TSetHeaZon 20 new_TSetHeaZon 19.5
19.5 None {'setpoint_': -1}
{'setpoint_': -1}
new shed counter 1
{'setpoint_x': 19.5}
ratchet ['setpoint_']
rebound []
rebound []
[]
